In [1]:
import json
import os
import pandas as pd
import csv
import keras
from numpy import array
import matplotlib.pyplot as plt
import subprocess

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

Selecting previously unselected package libgflags2.2.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [ ]:
cd openpose

/content/openpose


In [ ]:
# now we create landmarks for these videos
video_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/newVideo"
landmark_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/"
for items in os.listdir(video_path):  
  if len(items.split(".")) != 0:
    video_output_path = os.path.join(landmark_path, items)
    videoPath = os.path.join(video_path, items)
    command = './build/examples/openpose/openpose.bin --video "' + str(videoPath) +  '" --write_json "' + str(video_output_path) + '" --display 0 --render_pose 0 --net_resolution 1312x736 --scale_number 4 --scale_gap 0.25'
    x = subprocess.run(command, shell=True, capture_output=True)
    print(x.stdout.decode())

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 3022.871696 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 867.622760 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 1122.276453 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 2173.366546 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread

Starting here, we process the landmark data created by openpose on each video

In [3]:
# this function reads in all the landmarks, filter out all the landmarks without detecting any person, and return the data as a list
def readData(path):
    frameCounter = []
    emptyFrame = []
    totalData = []
    dataFrame = []
    for itemPath in os.listdir(path):
        names = itemPath.split("_")
        if len(names) < 4:
          continue      
        fullPath = os.path.join(path, itemPath)    
        Data = pd.read_json(fullPath) 
        pData = Data["people"]
        poseData = []
        if len(pData) != 0:  
          fullData = pData[0]["pose_keypoints_2d"]                  
          poseData.append(fullData[:45] + fullData[57:])
          dataFrame.append(itemPath.split('.')[0])
          totalData+=poseData
        else:
          emptyFrame.append(itemPath.split('.')[0])                
    return totalData,dataFrame, emptyFrame

In [4]:
# we need to normalize data
def normalizeData(totalData, width, height):
    for data in totalData:
        for i in range(0, 63, 3):
            data[i] = float(data[i]) / width
            data[i+1] = float(data[i+1]) / height
    return totalData

In [5]:
# this segment create a list that contains already processed videos
video_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/oldVideo"
landmark_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/"
csv_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData"
old_csv_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/oldCSVData"
existedData = []
for csvItem in os.listdir(csv_path):
  names = csvItem.split(".")
  if len(names) == 2:
    existedData.append(names[0])

for csvItem in os.listdir(old_csv_path):
  names = csvItem.split(".")
  if len(names) == 2:
    existedData.append(names[0])
print(existedData)

['test_19', 'test_20', 'test_21', 'test_22', 'test_23', 'test_24', 'test_25', 'test_26', 'test_27', 'test_28', 'test_29', 'test_30', 'test_31', 'test_10', 'test_08', 'test_09', 'test_03', 'test_07', 'test_05', 'test_04', 'test_06', 'test_02', 'test_01', 'test_17', 'test_15', 'test_14', 'test_18', 'test_13', 'test_12', 'test_11', 'test_16']


In [6]:
!ls "/content/drive/MyDrive/Colab Notebooks/Youtube/output/"

csvData				     test_23_000000002908_keypoints.json
oldCSVData			     test_23_000000002909_keypoints.json
test				     test_23_000000002910_keypoints.json
test_01				     test_23_000000002911_keypoints.json
test_02				     test_23_000000002912_keypoints.json
test_03				     test_23_000000002913_keypoints.json
test_04				     test_23_000000002914_keypoints.json
test_05				     test_23_000000002915_keypoints.json
test_06				     test_23_000000002916_keypoints.json
test_07				     test_23_000000002917_keypoints.json
test_08				     test_23_000000002918_keypoints.json
test_09				     test_23_000000002919_keypoints.json
test_10				     test_23_000000002920_keypoints.json
test_11				     test_23_000000002921_keypoints.json
test_12				     test_23_000000002922_keypoints.json
test_13				     test_23_000000002923_keypoints.json
test_14				     test_23_000000002924_keypoints.json
test_15				     test_23_000000002925_keypoints.json
test_16				     test_23_000000002926_keypoints.json
test_17				  

In [7]:

# this segment produces the csv file for each youtube video
for folderName in os.listdir(landmark_path):
  if len(folderName.split("_")) < 2:
    continue     
  if len(folderName.split(".")) > 1:
    continue
  if folderName in existedData:
    continue  
  
  folderPath = os.path.join(landmark_path, folderName)
  print(folderPath)
  totalData,dataFrame, emptyFrame = readData(folderPath)
  normalizedD = normalizeData(totalData, 1280, 720)
  df = pd.DataFrame.from_records(normalizedD)
  df.insert(63, "frameName", dataFrame)
  csvName = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/" + str(folderName) + ".csv"
  print("saving csv for " + csvName)
  df.head()
  df.to_csv(csvName)

/content/drive/MyDrive/Colab Notebooks/Youtube/output/test_32
saving csv for /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_32.csv
/content/drive/MyDrive/Colab Notebooks/Youtube/output/test_36
saving csv for /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_36.csv
/content/drive/MyDrive/Colab Notebooks/Youtube/output/test_35
saving csv for /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_35.csv
/content/drive/MyDrive/Colab Notebooks/Youtube/output/test_34
saving csv for /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_34.csv
/content/drive/MyDrive/Colab Notebooks/Youtube/output/test_33
saving csv for /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_33.csv


In [8]:
ls '/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/'

test_19.csv  test_22.csv  test_25.csv  test_28.csv  test_31.csv  test_34.csv
test_20.csv  test_23.csv  test_26.csv  test_29.csv  test_32.csv  test_35.csv
test_21.csv  test_24.csv  test_27.csv  test_30.csv  test_33.csv  test_36.csv


In [ ]:
!mv "/content/drive/MyDrive/Colab Notebooks/Youtube/output/test_20.csv" "/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData"

In [10]:
def sortData(videoName, basePath):
  path = os.path.join(basePath, videoName + ".csv")
  df = pd.read_csv(path)
  for index, row in df.iterrows():
    frameNumber = int(row[-1].split("_")[2])
    df.at[index, "frameName"] = frameNumber
  df.sort_values(by=["frameName"], inplace=True)
  return df

In [11]:
# now we do the prediction of the video
# function that returns the prediction of a video 
def predictData(df):
  twoSecondsSegments = []

  result = []

  df = df.iloc[:, 1:]
  frame = df.loc[:, 'frameName'].values
  df = df.drop(columns="frameName")  
  data = df.values
  print("number of data: " , len(data))
  # now we cut the video in to segments of 2 seconds clips
  frameNameTemp = None
  temp = []
  for i in range(len(data)):
    frameNumber = int(frame[i])
    if frameNameTemp is None:
      frameNameTemp = frameNumber
    temp.append(data[i])
    if (frameNumber - frameNameTemp) >= 60:
      twoSecondsSegments.append((frameNameTemp, frameNumber, temp))
      temp = []
      frameNameTemp = frameNumber + 1    
      
  model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/projectModelBILSTM.h5")  
  print("number of segments: " ,len(twoSecondsSegments))
  for item in twoSecondsSegments:
    x_test = array(item[2]).reshape(len(item[2]), 1, len(item[2][0]))          
    probability = model.predict(x_test)    
    counter = 0
    for p in probability:
      if p > 0.9:
        counter += 1
    if counter >= 30:
      probability = True
      result.append((item[0], item[1], probability, float(counter/60)))
  print("number of positive clips ", len(result))
  return result  


In [12]:
# this reads in the prediction by the model and produce the corresponding meta data
def createJson(result, videoID):
  data = []
  for item in result:
    startTime = float(item[0]) / 30
    endTime = float(item[1]) / 30
    newDict = {}
    newDict["videoId"] = str(vidoeID)
    newDict["type"] = "segment"
    newDict["startTime"] = startTime
    newDict["endTime"] = endTime
    newDict["observer"] = "CSCE636-Spring2021-MattDLSmartHome-v5"
    newDict["isHuman"] = False
    newDict["confirmedBySomeone"] = False
    newDict["rejectedBySomeone"] = False
    newDict["observation"] = {"label": "Fall", "labelConfidance": float(item[3])}     
    data.append(newDict)
  return data


  

In [13]:
# first read the video URL to extract the video ID
testURL = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Youtube/testURL.xlsx", header=0)
testInfo = {}
for index, row in testURL.iterrows():
  vidoeID = row[1].split('?')[1][2:]
  testInfo[row[0]] = vidoeID

In [14]:
# now we iterrate through all the test videos and produce the corresponding meta data
basePath = '/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData'
jsonPath = "/content/drive/MyDrive/Colab Notebooks/Youtube/json"
totalJson = []
for fileName in os.listdir(basePath):
  videoName = fileName.split(".")[0]
  print("procesing ", str(videoName))
  d = sortData(videoName, basePath)
  p = predictData(d)
  data = createJson(p, testInfo[videoName])
  print("for this clip we have: ")
  print(data)
  totalJson += data
  
path = os.path.join(jsonPath, "submission8.json")
with open(path, "w+") as jsonOutput:
    jsonOutput.write(json.dumps(totalJson))


procesing  test_19
number of data:  3796
number of segments:  71
number of positive clips  17
for this clip we have: 
[{'videoId': 'CPhDPQIgLjo', 'type': 'segment', 'startTime': 14.5, 'endTime': 16.533333333333335, 'observer': 'CSCE636-Spring2021-MattDLSmartHome-v5', 'isHuman': False, 'confirmedBySomeone': False, 'rejectedBySomeone': False, 'observation': {'label': 'Fall', 'labelConfidance': 0.6333333333333333}}, {'videoId': 'CPhDPQIgLjo', 'type': 'segment', 'startTime': 47.93333333333333, 'endTime': 49.93333333333333, 'observer': 'CSCE636-Spring2021-MattDLSmartHome-v5', 'isHuman': False, 'confirmedBySomeone': False, 'rejectedBySomeone': False, 'observation': {'label': 'Fall', 'labelConfidance': 0.7166666666666667}}, {'videoId': 'CPhDPQIgLjo', 'type': 'segment', 'startTime': 49.96666666666667, 'endTime': 51.96666666666667, 'observer': 'CSCE636-Spring2021-MattDLSmartHome-v5', 'isHuman': False, 'confirmedBySomeone': False, 'rejectedBySomeone': False, 'observation': {'label': 'Fall', 'lab

In [15]:
with open("/content/drive/MyDrive/Colab Notebooks/Youtube/json/submission8.json", "r") as jsonInput:
    t = json.load(jsonInput)
print(len(t))

312
